In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from qdrant_client.http import models

from qdrant_client import QdrantClient
from qdrant_client.http.models import CollectionDescription, VectorParams, Distance
import sqlite3
client = QdrantClient(host="sprakbankdb1.lx.nb.no", port=6333)


In [13]:
import numpy as np

In [33]:
collection_name = "vss_1850_cos"

In [32]:
db = "../Embeddings/vss_1850.db"

In [3]:
def query(db, sql, params=()):
    with sqlite3.connect(db) as con:
        r = con.execute(sql, params).fetchall()
    return r

In [10]:

def convert_vector(binary_vector):
    # Assuming binary_vector is a blob from SQLite, convert it to a list
    return np.frombuffer(binary_vector, dtype=np.float32).tolist()

In [68]:
res = query(db, "select * from words where word = 'Stadion' ")

query_vector = convert_vector(res[0][1])

hits = client.search(
   collection_name = collection_name,
   query_vector = query_vector,
   limit = 20  # Return 5 closest points
)

for x in hits:
    print(x.payload['word'], x.score)
    

Stadion 1.0
Schlick 0.77686065
Taxis 0.7519788
Thurn 0.745286
Overstaldmester 0.7417013
Wittgenstein 0.741445
Hohenlohe 0.73181033
Truchsess 0.7316167
Beust 0.7314053
Piccolomini 0.72863364
Soltikow 0.7273422
Antoniello 0.72576535
Altenstein 0.7250618
Galitzin 0.724588
Königsmark 0.7245572
Waldstein 0.7226002
Kniphausen 0.7213417
Wessenberg 0.72070366
Andrassy 0.71949124
Brunnow 0.71804744


In [64]:
try:
    count_response = client.count(collection_name=collection_name)
    total_points = count_response.count
    print(f"Total points in collection '{collection_name}': {total_points}")
except Exception as e:
    print(f"An error occurred: {e}")

Total points in collection 'vss_1850_cos': 2168213
